<a href="https://colab.research.google.com/github/MeghanGH/BigQuery-Python/blob/main/BigQuery_table_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery table creation

**[Big Data and Cloud Computing](https://moodle.up.pt/course/view.php?id=4854), Eduardo R. B. Marques, DCC/FCUP**

This notebook introduces you to the creation of BigQuery tables using the [__BigQuery client Python API__](https://googleapis.dev/python/bigquery/latest/).

For illustration we will consider the __'airports'__ and __'flights_airport'__ data sets that are part of  [__the Vega datasets__](https://github.com/vega/vega-datasets) that are acessible through the [__vega_datasets python package__](https://pypi.org/project/vega-datasets/).
The data at stake relates to airports and flights between airports in North America.



## Activate BigQuery for your GCP project

First make sure you access the [BigQuery user interface in the GCP console](https://console.cloud.google.com/bigquery) and activate BigQuery for the GCP project you will be using.

Keep the BigQuery UI open in your browser for checking the results of operations.

## Get access to Google Cloud from the notebook





### Set your project id

In [ ]:
PROJECT_ID = 'bdcc2023' #@param {type: "string"}


### Authenticate to Google Cloud

To authenticate to Google Cloud we will use the
a redirection to GCP that supplies a verification code.


In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


## Create BigQuery client

We are now ready to use BigQuery.

We first create the BigQuery client handle, i.e., a [Client](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client) object.

In [ ]:
import google.cloud.bigquery as bq

client = bq.Client(project=PROJECT_ID)

## Create Vega dataset in BigQuery

We will create the dataset first, so that we can associate tables to it. Running the code below, we obtain a [Dataset](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.dataset.Dataset.html) object.

After executing, check the BigQuery console interface to verify that the `vega` dataset exists. __Reload the page if necessary.__

In [ ]:
dataset = client.create_dataset('vega', exists_ok=True)

## Create the tables and associated schema



In [ ]:
import vega_datasets as vd
import pandas as pd

### Load airport data to use

In [ ]:
airports = pd.read_csv(vd.data.airports.url)
airports

,iata,name,city,state,country,latitude,longitude
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


### Create a corresponding table in BigQuery

Check the BigQuery interface after you run this step.

You should see a table called `airports` now.

In [ ]:
table_name = PROJECT_ID + '.vega.airports'
print('Creating table ' + table_name)

# Delete the table in case you're running this for the second time
client.delete_table(table_name, not_found_ok=True)

# Create the table
# - we use the same field names as in the original data set
table = bq.Table(table_name)
table.schema = (
        bq.SchemaField('iata',      'STRING'),
        bq.SchemaField('name',      'STRING'),
        bq.SchemaField('city',      'STRING'),
        bq.SchemaField('state',     'STRING'),
        bq.SchemaField('country',   'STRING'),
        bq.SchemaField('latitude',  'FLOAT' ),
        bq.SchemaField('longitude', 'FLOAT' )
)
client.create_table(table)

Creating table bdcc2023.vega.airports


Table(TableReference(DatasetReference('bdcc2023', 'vega'), 'airports'))

### Insert the data onto the BigQuery table

Check the BigQuery interface after you run this step.

You should see the `airports` table now has data.

__Confirm that it has the same number of rows as the Pandas dataframe.__

In [ ]:
import time

print('Loading data into ' + table_name)
load_job = client.load_table_from_dataframe(airports, table)

while load_job.running():
  print('waiting for the load job to complete')
  time.sleep(1)

if load_job.errors == None:
  print('Load complete!')
else:
  print(load_job.errors)

Loading data into bdcc2023.vega.airports
waiting for the load job to complete
waiting for the load job to complete
Load complete!


## Exercise. Do the same for flight data

At the end you should have created a table called `flights`.

__Confirm that it has the same number of rows as the Pandas dataframe.__

__Note:__ for the `count` field use the `INTEGER` type in the BigQuery schema.


In [ ]:
flights = pd.read_csv(vd.data.flights_airport.url)
flights

,origin,destination,count
0,ABE,ATL,853
1,ABE,BHM,1
2,ABE,CLE,805
3,ABE,CLT,465
4,ABE,CVG,247
...,...,...,...
5361,YUM,IPL,326
5362,YUM,LAS,99
5363,YUM,LAX,1044
5364,YUM,PHX,1961
